In [30]:
import cv2
import os
import mediapipe as mp
import numpy as np
import sklearn 
from sklearn import tree
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from os.path import isfile, join
import csv
import mediapipe as mp 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score



## Split des données 

In [31]:
df = pd.read_csv('/home/camille/main4/main4_capgemini/pre-traitement/données/stockage_csv/64_adresse_vouloir.csv')
nb_mots = 64

KeyboardInterrupt: 

In [ ]:
nb_frames = len(df.columns)-1
print(nb_frames)


6000


In [ ]:
#remplace nan with O
df =df.fillna(0)

#print(df.isnull().sum())

#on split les données en train et test
Y = df[['class']]
X = df.iloc[:, 1:6001]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=27, stratify=Y)



## KNN - K plus proches voisins

In [ ]:
#on utilise le classificateur KNN
knn = KNeighborsClassifier(n_neighbors=nb_mots, leaf_size=100)

#Train the model on the training data
knn.fit(X_train, y_train)

#Make predictions on the test data
y_pred = knn.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

#1 donc tout est bon !


# Calcul de la précision
precision = precision_score(y_test, y_pred, average='micro')

# Calcul du rappel
recall = recall_score(y_test, y_pred, average='micro')

# Calcul de l'F1-score
f1 = f1_score(y_test, y_pred, average='micro')

# Calcul de l'AUC
#auc = roc_auc_score(y_test, y_pred, average='micro')

print(precision,recall,f1)



0.4357142857142857 0.4357142857142857 0.4357142857142857


In [ ]:

# Calcul de la précision
precision = precision_score(y_test, y_pred, average='micro')

# Calcul du rappel
recall = recall_score(y_test, y_pred, average='micro')

# Calcul de l'F1-score
f1 = f1_score(y_test, y_pred, average='micro')

# Calcul de l'AUC
#auc = roc_auc_score(y_test, y_pred, average='micro')

print(precision,recall,f1)

0.7870370370370371 0.7870370370370371 0.7870370370370371


## arbres de décisions


In [ ]:
model_tree = tree.DecisionTreeClassifier()
model_tree = model_tree.fit(X_train, y_train)


# print(model_tree.predict(X_test))
# print(y_test)

sklearn.metrics.accuracy_score(y_test, model_tree.predict(X_test))

y_pred = model_tree.predict(X_test)

# Calcul de la précision
precision = precision_score(y_test, y_pred, average='micro')

# Calcul du rappel
recall = recall_score(y_test, y_pred, average='micro')

# Calcul de l'F1-score
f1 = f1_score(y_test, y_pred, average='micro')

# Calcul de l'AUC
#auc = roc_auc_score(y_test, y_pred, average='micro')

print(precision,recall,f1)


0.8148148148148148 0.8148148148148148 0.8148148148148148


YOUHOU premiers résultats superbes


## Stochastic Gradient Descent
 

In [ ]:
clf_SGD = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)

clf_SGD.fit(X_train, y_train)

SGDClassifier(max_iter=5)


# print(y_test)

sklearn.metrics.accuracy_score(y_test, clf_SGD.predict(X_test))

0.2222222222222222

## Support vector machines


In [ ]:
clf_SVM = svm.SVC()
clf_SVM.fit(X_train, y_train)


sklearn.metrics.accuracy_score(y_test, clf_SVM.predict(X_test))
y_pred = clf_SVM.predict(X_test)

# Calcul de la précision
precision = precision_score(y_test, y_pred, average='micro')

# Calcul du rappel
recall = recall_score(y_test, y_pred, average='micro')

# Calcul de l'F1-score
f1 = f1_score(y_test, y_pred, average='micro')

# Calcul de l'AUC
#auc = roc_auc_score(y_test, y_pred, average='micro')

print(precision,recall,f1)

/home/camille/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6083333333333333 0.6083333333333333 0.6083333333333333


## MPL


In [ ]:
clf_neural = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(15,), random_state=1)
clf_neural.fit(X_train, y_train)
sklearn.metrics.accuracy_score(y_test, clf_neural.predict(X_test))




0.7222222222222222

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Définition des paramètres à tester
parameters = {
    'hidden_layer_sizes': [(10,), (20,)],
    'alpha': [0.001, 0.01],
    'max_iter': [100, 200]
}

# Initialisation de l'objet GridSearchCV
mlp = MLPClassifier()
clf = GridSearchCV(mlp, parameters, scoring='accuracy', cv=5)

# Entrainement du classificateur
clf.fit(X_train, y_train)

# Affichage des meilleurs paramètres
print("Les meilleurs paramètres sont:", clf.best_params_)
print("La meilleure précision est:", clf.best_score_)

On voit que ça marche moins bien, voir si les paramètres sont bien réglés

# Détection en temps réel

In [ ]:
mp_drawing = mp.solutions.drawing_utils  # Drawing helpers
mp_holistic = mp.solutions.holistic  # Mediapipe Solutions

def mediapipe_detection(image, model): #Fonction pour que mediapipe puisse détecter (car il ne travaille pas dans le même système de couleur que OpenCV)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
                            
def extract_keypoints(results):
    pose = (np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4))
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = (np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3))
    rh = (np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3))
    return np.concatenate([pose, lh, rh])

from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [37]:
# 1. New detection variables
actions = np.array(['adresse','affaire','aller','ami_amie','autre','beaucoup','bonjour','chocolat','comprendre','demander','dieu','donc','dormir','enceinte','faire','famille','finir','gens','heure','ils_elles','interdire','jamais','jour','laisser','lentement','marcher_marche','merci','mourir_mort','nous_on','nuit','ou','payer','penser','personne','peu','pleuvoir_pluie','portugal','pour','prendre','question','quoi','raison_connaissance','rencontrer_rencontre','rester','rien','rue_route','sac','soeur_nonne','soleil','surprendre_surprise','temps','toujours','travail','trop','trouver','turquie','vache','vivre_vie','voir','vouloir'])
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
       # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-nb_frames:]
        
        if len(sequence) == nb_frames:
            res = clf_SVM.predict(sequence)
        #     predictions.append(np.argmax(res))
            
            
        # #3. Viz logic
        #     if np.unique(predictions[-10:])[0]==np.argmax(res): 
        #         if res[np.argmax(res)] > threshold: 
                    
        #             if len(sentence) > 0: 
        #                 if actions[np.argmax(res)] != sentence[-1]:
        #                     sentence.append(actions[np.argmax(res)])
        #             else:
        #                 sentence.append(actions[np.argmax(res)])

        #     if len(sentence) > 5: 
        #         sentence = sentence[-5:]

        #     # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(res), (3,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 